In [9]:
# using NLopt
using Optim
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl");

Ok, so I think the way this works is that `myfunc` is going to supply the update in the gradient by modifying it in place. This seems very smart -- each function call updates the gradient. So, what we would need to do evaluate the relevant sums and cut terms, then follow this pattern. 

In [134]:
# parameters

n = 100
Z = rand(1:10, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = maximum(1.0*p).^α[1] / (sum((1.0*p).^α[1])*α[2])

α0 = [100.0, 1000]

kmax = 3

Ω = buildΩ(ω, α0, kmax)

Ω (generic function with 1 method)

In [135]:
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [51, 70, 63, 62, 90, 71, 42, 39, 66, 49  …  68, 60, 60, 51, 52, 84, 63, 62, 56, 48]


In [136]:
function obj(x)
    """
    objective for optimization. Look into gradients later. 
    """
    Q = modularity(H::hypergraph, Z::Array{Int64,1}, Ω; α=x, bigInt=true)
#     println(Float64(Q))
    return(Float64(-Q))
end

obj (generic function with 1 method)

In [137]:
res = optimize(obj, 0, [10.0, 1000.0],
    NelderMead(),
    Optim.Options(f_tol = 1e-10,
        iterations = 1000,
        show_trace=false))
α̂ = Optim.minimizer(res)

2-element Array{Float64,1}:
 6.823825797767675
 5.654428282239759e7

In [138]:
Optim.minimum(res)

38966.26143714044

In [139]:
# true
logLikelihood(H, Z, Ω, ϑ; α=α0) 

-10639.04564657328

In [140]:
# before estimating α
logLikelihood(H, Z, Ω, 1.0*H.D; α=α0)

-Inf

In [141]:
# after estimating α
logLikelihood(H, Z, Ω, 1.0*H.D; α=α̂)

-10642.300740435123

In [142]:
α̂ = α0
Ẑ = Z
for i =1:5
    Z_prev = Ẑ
    Ẑ = HyperLouvain(H,kmax,Ω;α=α̂)
#     println(Z_prev==Ẑ)
    
    function obj(x)
        """
        objective for optimization. Look into gradients later. 
        """
        Q = modularity(H, Ẑ, Ω; α=x, bigInt=true)
        return(Float64(-Q))
    end
    
    res = optimize(obj, 0, [10.0, 100.0],
            NelderMead(),
            Optim.Options(f_tol = 1e-10,
                iterations = 1000,
                show_trace=false))
    print(-Optim.minimum(res))
    α_prev = α̂
    α̂ = Optim.minimizer(res) 
end


Louvain Iteration 1
No nodes moved clusters
-39236.28444138926
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
-39012.77153062707
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
Louvain Iteration 8
Louvain Iteration 9
Louvain Iteration 10
-39000.52400653719
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
Louvain Iteration 8
Louvain Iteration 9
Louvain Iteration 10
-885828.5158337515
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
-39350.52444521978

In [144]:
α̂

2-element Array{Float64,1}:
 0.28100945132157384
 5.8037980560648486e7

In [145]:
length(unique(Ẑ)) # number of clusters

3